# Deeplearning LSTM Model Training and Testing
In the notebook `ml/deeplearning_tuning.ipynb` we performed a Bayesian hyperparameter tuning to find the best hyperparameters for 3 potential LSTM models architectures.
In this notebook we train these 3 different LSTM model architectures with the best hyperparameters, to try to predict the target variable based on the historical data of all the features.
We then perform a analysis of the sensitivity of the best model on the hyperparameter values.

In [7]:
import sys
sys.path.append('..')

In [8]:
import os
import pickle
import numpy as np
import pandas as pd
import random
import tensorflow

from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, RepeatVector
from keras.optimizers import Adam
from lib.read_data import read_and_join_output_file
from lib.deeplearning import get_train_test_datasets,  get_sets_shapes, evaluate_forecast
from lib.viz import draw_hyperparameters_distribution

In [3]:
RANDOM_SEED = 31
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tensorflow.random.set_seed(RANDOM_SEED)

In [4]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Preparing the Dataset
### The Train-Test Split
The dataset is made of 478 Township-Ranges, each containing a multivariate (81 features) time series (data between 2014 to 2021). This dataset can thus be seen as a 3 dimensional dataset of
$478 TownshipRanges * 8 time stamps * 81 features$
The objective is to predict the 2022 target value of `GSE_GWE` (Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface) for each Township-Range.

![Multi-Variate Multi TImes-Series Predictions with LSTM - Training and Prediction](../doc/images/lstm_inputs_outputs.jpg)

LSTM neural networks can be used for time series forecasting and take inputs of the shape *[samples, time series steps, features]*. This perfectly fits our dataset.
To fit our dataset and objective, as well as LSTM neural networks architecture we perform the train test split as follow:
* Training and Test sets will be split by Township-Ranges. I.e., some Township-Ranges will have all their 2014-2021 data points in the training set, some others will be in the test set.
* The model will be trained based on the 2014-2020 data for all features - including the target feature - and will be trained and tested on the 2021 value of the target feature.

With such a method, unlike a simple time series forecasting where the target feature is forecasted only based on its past value, we allow past value of other features (in our case cultivated crops, precipitations, population density, number of wells drilled) to influence the future value of the target feature.

![Train-Test Split](../doc/images/lstm-train-test-split.jpg)

We do not create a validation dataset as we use Keras internal cross-validation mechanism to shuffle the data points (i.e., the Township-Ranges) and keep some for the validation at each training epoch.
### Data Imputation and Scaling
Missing data imputation for a Township-Range is performed only using the existing data of that Township-Range (not the data of all Township-Ranges). For example:
* a *fill forward* approach is used for many fields like crops, vegetation and soils. The percentage of land use per crop in 2014 in a Township-Range is imputed into the missing year 2015 for that particular Township-Range.
* for fields like `PCT_OF_CAPACITY` (the capacity percentage of water reservoir), missing values in a Township-Range are filled using the min, mean, median or max values of that particular Township-Range
This approach means that the data imputation *fit* method does not need to learn values from other Township-Ranges data points to impute missing values. Since our train and test datasets are split by Township-Ranges, it avoids issues when  using the impute pipeline fitted on the training dataset to impute data for Township-Ranges the impute pipeline has not seen before.

We use a MinMax scaler to scale all values between 0 and 1 for the neural network.

It should be noted that we do not need to do any data imputation on the training and test sets *y* target feature since it does not have any missing data point.

In [5]:
test_size=0.15
target_variable="GSE_GWE"
# Load the data from the ETL output files
X = read_and_join_output_file()
# Split the input pandas Dataframe into training and test datasets, applies the impute pipeline
# transformation and reshapes the datasets to 3D (samples, time, features) numpy arrays
X_train, X_test, y_train, y_test, impute_pipeline, target_scaler = get_train_test_datasets(X, target_variable=target_variable,
    test_size=test_size, random_seed=RANDOM_SEED, save_to_file=True)
model_predictions_df = pd.DataFrame(y_test, columns=[target_variable])
model_scores_df = pd.DataFrame(columns=["mae", "mse", "rmse"])
nb_features = X_train.shape[-1]
get_sets_shapes(X_train, X_test)

,nb_items,nb_timestamps,nb_features
training dataset,406,7,81
test dataset,72,7,81


## Training Different Models
We tried 3 different LSTM models:
* A simple model made of a single *LSTM* layer and an output *Dense* layer
* A model made of a *LSTM* layer followed by a *Dense* and *Dropout* layers before the output layer
* An Encoder-Decoder model made of 2 *LSTM* layers followed by a *Dense* and *Dropout* layers

![LSTM Model Architectures](../doc/images/lstm_architectures.jpg)


Encoder-decoder architectures are more common for sequence to sequence learning e.g., when forecasting the next 3 days (output sequence of length 3) based on the past year data (input sequence of length 365). In our case we only predict data for 1 time step in the feature. The output sequence being of length 1 this architecture might seem superfluous but has been tested anyway. This architecture was inspired by the Encoder-Decoder architecture in this article: *[CNN-LSTM-Based Models for Multiple Parallel Input and Multi-Step Forecast](https://towardsdatascience.com/cnn-lstm-based-models-for-multiple-parallel-input-and-multi-step-forecast-6fe2172f7668)*.

As such models are made for sequence to sequence learning and forecasting, the output of such a model is different from the previous ones. It has an output of size *[samples, forecasting sequence length, target features]*. In our case the forecasting sequence length and number of target features are both 1.
## Training Model 1 - Simple LSTM Model

In [6]:
m1_hyper_parameters = {
    "lstm_units": 160,
    "lstm_activation": "sigmoid",
    "learning_rate": 0.001,
    "validation_split": 0.1,
    "batch_size": 128,
    "epochs": 270,
}

model1 = Sequential()
model1.add(LSTM(m1_hyper_parameters["lstm_units"], activation=m1_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model1.add(Dense(1, activation="linear"))
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 160)               154880    
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 155,041
Trainable params: 155,041
Non-trainable params: 0
_________________________________________________________________


In [7]:
model1.compile(loss="mse", optimizer=Adam(learning_rate=m1_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model1.fit(X_train, y_train,
           validation_split=m1_hyper_parameters["validation_split"],
           batch_size=m1_hyper_parameters["batch_size"],
           epochs=m1_hyper_parameters["epochs"],
           shuffle=True)
yhat = model1.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat)
model_predictions_df["model_1_prediction"] = yhat_inverse
model_scores_df.loc["model 1"] = evaluate_forecast(y_test, yhat_inverse)

Epoch 1/270
3/3 [==============================] - 1s 132ms/step - loss: 0.1125 - root_mean_squared_error: 0.3354 - val_loss: 0.1359 - val_root_mean_squared_error: 0.3686
Epoch 2/270
3/3 [==============================] - 0s 25ms/step - loss: 0.0867 - root_mean_squared_error: 0.2945 - val_loss: 0.0269 - val_root_mean_squared_error: 0.1641
Epoch 3/270
3/3 [==============================] - 0s 21ms/step - loss: 0.0568 - root_mean_squared_error: 0.2384 - val_loss: 0.0396 - val_root_mean_squared_error: 0.1990
Epoch 4/270
3/3 [==============================] - 0s 21ms/step - loss: 0.0579 - root_mean_squared_error: 0.2406 - val_loss: 0.0281 - val_root_mean_squared_error: 0.1676
Epoch 5/270
3/3 [==============================] - 0s 21ms/step - loss: 0.0434 - root_mean_squared_error: 0.2082 - val_loss: 0.0568 - val_root_mean_squared_error: 0.2383
Epoch 6/270
3/3 [==============================] - 0s 21ms/step - loss: 0.0466 - root_mean_squared_error: 0.2158 - val_loss: 0.0306 - val_root_mean_s

## Training Model 2 - LSTM + Dense Layer Model

In [8]:
m2_hyper_parameters = {
    "lstm_units": 100,
    "lstm_activation": "sigmoid",
    "dense_units": 11,
    "dense_activation": "tanh",
    "dropout": 0.1,
    "learning_rate": 0.0001,
    "validation_split": 0.1,
    "batch_size": 32,
    "epochs": 200,
}

model2 = Sequential()
model2.add(LSTM(m2_hyper_parameters["lstm_units"], activation=m2_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model2.add(Dense(m2_hyper_parameters["dense_units"], activation=m2_hyper_parameters["dense_activation"]))
model2.add(Dropout(m2_hyper_parameters["dropout"]))
model2.add(Dense(1, activation="linear"))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               72800     
                                                                 
 dense_1 (Dense)             (None, 11)                1111      
                                                                 
 dropout (Dropout)           (None, 11)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 12        
                                                                 
Total params: 73,923
Trainable params: 73,923
Non-trainable params: 0
_________________________________________________________________


In [9]:
model2.compile(loss="mse", optimizer=Adam(learning_rate=m2_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model2.fit(X_train, y_train,
           validation_split=m2_hyper_parameters["validation_split"],
           batch_size=m2_hyper_parameters["batch_size"],
           epochs=m2_hyper_parameters["epochs"],
           shuffle=True)
yhat = model2.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat)
model_predictions_df["model_2_prediction"] = yhat_inverse
model_scores_df.loc["model 2"] = evaluate_forecast(y_test, yhat_inverse)

Epoch 1/200
12/12 [==============================] - 1s 25ms/step - loss: 0.1466 - root_mean_squared_error: 0.3829 - val_loss: 0.0319 - val_root_mean_squared_error: 0.1786
Epoch 2/200
12/12 [==============================] - 0s 8ms/step - loss: 0.1169 - root_mean_squared_error: 0.3419 - val_loss: 0.0270 - val_root_mean_squared_error: 0.1644
Epoch 3/200
12/12 [==============================] - 0s 7ms/step - loss: 0.1023 - root_mean_squared_error: 0.3199 - val_loss: 0.0341 - val_root_mean_squared_error: 0.1846
Epoch 4/200
12/12 [==============================] - 0s 7ms/step - loss: 0.0963 - root_mean_squared_error: 0.3103 - val_loss: 0.0341 - val_root_mean_squared_error: 0.1847
Epoch 5/200
12/12 [==============================] - 0s 7ms/step - loss: 0.0990 - root_mean_squared_error: 0.3146 - val_loss: 0.0319 - val_root_mean_squared_error: 0.1787
Epoch 6/200
12/12 [==============================] - 0s 7ms/step - loss: 0.0878 - root_mean_squared_error: 0.2964 - val_loss: 0.0300 - val_root_

## Training Model 3 - Encoder-Decoder LSTM Model

In [10]:
m3_hyper_parameters = {
    "lstm_units": 300,
    "lstm_activation": "sigmoid",
    "2nd_lstm_units": 140,
    "2nd_lstm_activation": "sigmoid",
    "dense_units": 21,
    "dense_activation": "tanh",
    "dropout": 0.2,
    "learning_rate": 0.001,
    "validation_split": 0.1,
    "batch_size": 32,
    "epochs": 200,
}


model3 = Sequential()
model3.add(LSTM(m3_hyper_parameters["lstm_units"], activation=m3_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model3.add(RepeatVector(1))
model3.add(LSTM(m3_hyper_parameters["2nd_lstm_units"], activation=m3_hyper_parameters["lstm_activation"], return_sequences=True))
model3.add(TimeDistributed(Dense(m3_hyper_parameters["dense_units"], activation=m3_hyper_parameters["dense_activation"])))
model3.add(Dropout(m3_hyper_parameters["dropout"]))
model3.add(Dense(1, activation="linear"))
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 300)               458400    
                                                                 
 repeat_vector (RepeatVector  (None, 1, 300)           0         
 )                                                               
                                                                 
 lstm_3 (LSTM)               (None, 1, 140)            246960    
                                                                 
 time_distributed (TimeDistr  (None, 1, 21)            2961      
 ibuted)                                                         
                                                                 
 dropout_1 (Dropout)         (None, 1, 21)             0         
                                                                 
 dense_4 (Dense)             (None, 1, 1)             

In [11]:
y_train_3d =  y_train[..., np.newaxis]
model3.compile(loss="mse", optimizer=Adam(learning_rate=m3_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model3.fit(X_train, y_train_3d,
           validation_split=m3_hyper_parameters["validation_split"],
           batch_size=m3_hyper_parameters["batch_size"],
           epochs=m3_hyper_parameters["epochs"],
           shuffle=True)
yhat = model3.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat.squeeze(2))
model_predictions_df["model_3_prediction"] = yhat_inverse
model_scores_df.loc["model 3"] = evaluate_forecast(y_test, yhat_inverse)

Epoch 1/200
12/12 [==============================] - 2s 43ms/step - loss: 0.1867 - root_mean_squared_error: 0.4321 - val_loss: 0.0250 - val_root_mean_squared_error: 0.1581
Epoch 2/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0832 - root_mean_squared_error: 0.2885 - val_loss: 0.0322 - val_root_mean_squared_error: 0.1795
Epoch 3/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0597 - root_mean_squared_error: 0.2444 - val_loss: 0.0423 - val_root_mean_squared_error: 0.2057
Epoch 4/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0444 - root_mean_squared_error: 0.2107 - val_loss: 0.0344 - val_root_mean_squared_error: 0.1854
Epoch 5/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0387 - root_mean_squared_error: 0.1968 - val_loss: 0.0273 - val_root_mean_squared_error: 0.1653
Epoch 6/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0403 - root_mean_squared_error: 0.2008 - val_loss: 0.0283 - val_

## Comparing the Different Models
### Comparing the Model Scores

In [12]:
model_scores_df

,mae,mse,rmse
model 1,23.666956,1212.567017,34.821934
model 2,29.587023,1729.227051,41.583977
model 3,30.181566,1613.689209,40.170750


### Comparing the Model Predictions on the Test Dataset
Here we are comparing the target variable values for the year 2021 for the Township-Ranges in the test set compared to the prediction made by each model based on the 2014-2020 data for the Township-Ranges in the test set.

In [13]:
model_predictions_df

,GSE_GWE,model_1_prediction,model_2_prediction,model_3_prediction
0,33.198000,24.226774,28.419720,8.400249
1,34.795000,50.136299,54.256641,31.891882
2,161.756667,67.704063,69.008156,48.474617
3,54.423000,41.642086,41.073723,21.912615
4,80.653077,96.102310,102.622726,73.064133
...,...,...,...,...
67,187.252308,179.061325,164.998611,167.344238
68,179.551290,154.905746,162.471863,149.055557
69,236.543750,248.222504,249.016388,241.518127
70,292.550000,274.849457,250.053467,255.379242


Based on the model scores it turns out that the simplest of the three LSTM models is the one having the best scores.

However considering all the measurements between 2014 and 2022, the `GSE_GWE` (Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface) target value has a
* median of 137.09 (~41.7 meters)
* mean value of 167.37 feet (~50.9 meters)
* min value of 0.5 feet (0 meters)
* max value of 727.5 feet (221.6 meters)

A mean average error of 23.66 feet (7.2 meters), and root mean square error of 34.82 feet (10.6 meters) in the prediction is fairly large. Even the best model does not seem to be accurate enough to be useful.

We save the best model anyway to perform predictions and analyze the results. Refer to the notebook `/ml/deeplearning_results.ipynb` for the analysis of the 2022 predictions results.

In [14]:
model_dir = "../assets/models/"
keras_model_dir = os.path.join(model_dir, "keras_lstm_model")
os.makedirs(keras_model_dir, exist_ok=True)
# Save the Keras Model
model1.save(keras_model_dir)
# Save the data imputation pipeline and target min-max scaler
pipeline_data = {
    "impute_pipeline": impute_pipeline,
    "target_scaler": target_scaler
}
with open(os.path.join(model_dir, "lstm_model_pipeline.pkl"), "wb") as file:
    pickle.dump(pipeline_data, file)

INFO:tensorflow:Assets written to: ../assets/models/keras_lstm_model\assets


## Sensitivity Analysis
We perform here an analysis of the best model's sensitivity to the following hyperparamters:
* the optimizer used (e.g. Adam RMSprop, Adagrad)
* the training validation datasets split
* the number of lstm units
* the learning rate
* the batch size
* the number of training epochs

To perform this analysis we trained 33,344 LSTM models for all possible combinations of those 6 hyperparameters on the best model, and recorded for each model, the Root Mean Square Error (RMSE) on the test set.
The results are stored in a CSV file available in the ../assets/tuning folder.

The below visualization displays for each hyperparameter value, the distribution of the RMSE, and the mean of RMSE (using the color), for all models trained with that hyperparameter value. This allows us to show if a specific hyperparameter tends to lead to lower or higher RMSE and to compare the distribution between two values of the same hyperparameters.

In [10]:
hpt_df = pd.read_csv(r"../assets/tuning/hpt_results.csv")
# We can discard some hyperparameter data to reduce the size of the visualization and improve readability
#hpt_df = hpt_df[hpt_df["epochs"].isin(range(50, 310, 40))]
#hpt_df = hpt_df[hpt_df["lstm_units"].isin(range(10, 200, 30))]
draw_hyperparameters_distribution(hpt_df, ["optimizer", "validation_split", "learning_rate", "batch_size", "epochs", "lstm_units"], legend_y_pos=250)

alt.HConcatChart(...)

Looking at this visualization, we can see, with some surprise, that the hyperpareters which seem to have the biggest impact on the model performance have little to do with the model architecture itself (the number of LSTM units) but with how the model is trained.